# Report "MovieLens analytics"

## Структура отчета
1. Введение: цель, ограничения, описание данных
2. Объекты анализа: инициализация классов Movies, Ratings, Tags, Links
3. Фильмы: распределения по годам релиза и жанрам, топ по числу жанров
4. Рейтинги и фильмы: распределения по годам и значениям рейтингов, топы и спорные фильмы
5. Пользователи: активность, распределение средней/медианной оценки, топ по дисперсии
6. Теги: популярные теги, самые длинные, самые многословные, поиск по слову
7. Links/IMDb: получение полей по фильмам через IMDb и расчет топов
8. Выводы

## 1. Введение: цель, ограничения, описание данных

### Цель отчета
Цель - построить небольшой аналитический рассказ по данным `MovieLens`:
    
* понять, какие фильмы представлены

* посмотреть, как пользователи оценивают анализируемые нами фильмы и оценить поведение пользователей

* изучить теги, текстовые описания фильмов

* подготовить базу для расширения анализа через внешние метаданные IMDb/TMDb, используя связи из links.csv.

Отчет выполняется только с помощью методов нашего модуля `movielens_analysis.py`, а производительность каждого шага измеряется через `%timeit`.

### Информация о датасете
Используется датасет MovieLens, содержащий оценки и теги пользователей сервиса рекомендаций фильмов. Данные разбиты на четыре сsv файла:
* `movies.csv`: movieId, title, genres. Жанры перечислены через |. В названии обычно указан год релиза в скобках (например, Toy Story (1995)), но возможны пропуски/ошибки.
* `ratings.csv`: userId, movieId, rating, timestamp. Оценки 0 от 0.5 до 5.0 с шагом 0.5. timestamp - Unix time (секунды с 01.01.1970 UTC). В исходном датасете строки отсортированы по userId, затем movieId.
* `tags.csv`: userId, movieId, tag, timestamp. Теги - пользовательские короткие слова или фразы, отражающие ассоциации или описание фильма.
* `links.csv`: movieId, imdbId, tmdbId. Файл связывает идентификаторы MovieLens с внешними базами: imdbId позволяет строить ссылку вида https://www.imdb.com/title/tt<imdbId>/, а tmdbId - ссылку на TheMovieDB. Эти связи можно использовать для расширения анализа внешними характеристиками фильмов (например, режиссер, бюджет, мировые сборы, длительность).

Информация была взята из readme датасета.

По требованиям проекта анализ выполняется на первых 1000 строках каждого файла (после заголовка), чтобы ускорить расчеты и сделать результаты воспроизводимыми.

## 2. Объекты анализа: инициализация классов Movies, Ratings, Tags, Links

Импортируем только наш модуль:

In [1]:
import movielens_analysis as mla

Инициализируем наши классы и начнем работать с исходными данными датасета:

In [2]:
%timeit -r1 -n1 mla.Movies("movies.csv")
movies = mla.Movies("movies.csv")
movies.data[:3]

40 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


[{'movieId': 1,
  'title': 'Toy Story (1995)',
  'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
  'year': 1995},
 {'movieId': 2,
  'title': 'Jumanji (1995)',
  'genres': ['Adventure', 'Children', 'Fantasy'],
  'year': 1995},
 {'movieId': 3,
  'title': 'Grumpier Old Men (1995)',
  'genres': ['Comedy', 'Romance'],
  'year': 1995}]

In [3]:
%timeit -r1 -n1 mla.Ratings("movies.csv", "ratings.csv")
ratings = mla.Ratings("movies.csv", "ratings.csv")
ratings.data[:3]

27.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


[{'userId': 1,
  'movieId': 1,
  'rating': 4.0,
  'timestamp': 964982703,
  'title': 'Toy Story (1995)',
  'year': 1995,
  'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']},
 {'userId': 1,
  'movieId': 3,
  'rating': 4.0,
  'timestamp': 964981247,
  'title': 'Grumpier Old Men (1995)',
  'year': 1995,
  'genres': ['Comedy', 'Romance']},
 {'userId': 1,
  'movieId': 6,
  'rating': 4.0,
  'timestamp': 964982224,
  'title': 'Heat (1995)',
  'year': 1995,
  'genres': ['Action', 'Crime', 'Thriller']}]

In [4]:
%timeit -r1 -n1 mla.Tags("tags.csv")
tags = mla.Tags("tags.csv")
tags.tags[:3]

7.35 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


[{'userId': 2, 'movieId': 60756, 'tag': 'funny', 'timestamp': 1445714994},
 {'userId': 2,
  'movieId': 60756,
  'tag': 'Highly quotable',
  'timestamp': 1445714996},
 {'userId': 2,
  'movieId': 60756,
  'tag': 'will ferrell',
  'timestamp': 1445714992}]

In [5]:
%timeit -r1 -n1 mla.Links("links.csv",100)
links = mla.Links("links.csv",100)
links.data[:3]

803 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


[{'movieId': 1, 'imdbId': '0114709', 'tmdbId': '862'},
 {'movieId': 2, 'imdbId': '0113497', 'tmdbId': '8844'},
 {'movieId': 3, 'imdbId': '0113228', 'tmdbId': '15602'}]

## 3. Фильмы: распределения по годам релиза и жанрам, топ по числу жанров

#### Какие годы релиза чаще всего встречаются среди первых 1000 фильмов?

Метод `movies.dist_by_release()` считает год релиза из title и возвращает словарь {год: количество}. Словарь отсортирован по убыванию количества.

In [7]:
%timeit -r1 -n1 movies.dist_by_release()
movies.dist_by_release()

700 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{1995: 224,
 1994: 184,
 1996: 181,
 1993: 101,
 1992: 23,
 1990: 15,
 1991: 15,
 1989: 14,
 1986: 9,
 1982: 8,
 1940: 8,
 1957: 8,
 1987: 8,
 1980: 8,
 1981: 7,
 1988: 7,
 1979: 7,
 1955: 6,
 1959: 6,
 1968: 6,
 1997: 6,
 1939: 6,
 1985: 6,
 1967: 5,
 1965: 5,
 1951: 5,
 1958: 5,
 1944: 5,
 1941: 5,
 1975: 5,
 1971: 5,
 1984: 5,
 1964: 4,
 1973: 4,
 1954: 4,
 1934: 4,
 1960: 4,
 1963: 4,
 1950: 4,
 1974: 4,
 1983: 4,
 1977: 3,
 1937: 3,
 1972: 3,
 1952: 3,
 1961: 3,
 1953: 3,
 1946: 3,
 1938: 3,
 1956: 3,
 1962: 3,
 1976: 2,
 1969: 2,
 1970: 2,
 1942: 2,
 1945: 2,
 1947: 2,
 1935: 2,
 1936: 2,
 1949: 2,
 1978: 2,
 1943: 1,
 1932: 1,
 1966: 1,
 1948: 1,
 1933: 1,
 1931: 1}

**Вывод:**

Больше всего фильмов вышло в 1995 году.

#### Какие жанры встречаются чаще всего среди первых 1000 фильмов?

Метод `movies.dist_by_genres()` собирает все жанры из поля genres и возвращает {жанр: количество} (сортировка по убыванию количества).

In [8]:
%timeit -r1 -n1 movies.dist_by_genres()
movies.dist_by_genres()

1.86 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'Drama': 507,
 'Comedy': 365,
 'Romance': 208,
 'Thriller': 179,
 'Action': 158,
 'Adventure': 126,
 'Crime': 122,
 'Children': 100,
 'Fantasy': 69,
 'Sci-Fi': 69,
 'Mystery': 58,
 'Musical': 53,
 'Horror': 51,
 'War': 48,
 'Animation': 37,
 'Documentary': 25,
 'Western': 23,
 'Film-Noir': 18,
 'IMAX': 3}

**Вывод:**

Самые часто встречающиеся жанры это - романтика, триллер и боевик.

#### Какие фильмы самые многожанровые (имеют больше всего жанров)?

Метод `movies.most_genres(n)` считает количество жанров у каждого фильма и возвращает топ n как словарь {title: count_genres}. При равенстве количества жанров используется дополнительная сортировка по названию

In [9]:
%timeit -r1 -n1 movies.most_genres(10)
movies.most_genres(10)

1.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'Aladdin and the King of Thieves (1996)': 6,
 'All Dogs Go to Heaven 2 (1996)': 6,
 'Beauty and the Beast (1991)': 6,
 'Getaway, The (1994)': 6,
 'Lion King, The (1994)': 6,
 'Space Jam (1996)': 6,
 'Strange Days (1995)': 6,
 'Super Mario Bros. (1993)': 6,
 'Aladdin (1992)': 5,
 'Alice in Wonderland (1951)': 5}

**Вывод:**

Больше всего жанров у 6 фильмов:
'Aladdin and the King of Thieves (1996)', 'All Dogs Go to Heaven 2 (1996)', 'Beauty and the Beast (1991)',
 'Getaway, The (1994)',
 'Lion King, The (1994)',
 'Space Jam (1996)',
 'Strange Days (1995)',
 'Super Mario Bros. (1993)'.


Все фильмы с большим количеством жанров вышли до 2000 года.

## 4. Рейтинги и фильмы: распределения по годам и значениям рейтингов, топы и спорные фильмы


#### В какие годы пользователи чаще всего оставляли оценки?

Метод `ratings.class_movies.dist_by_year()` берет год и возвращает {год: количество оценок} с сортировкой по возрастанию года.

In [10]:
%timeit -r1 -n1 ratings.class_movies.dist_by_year()
ratings.class_movies.dist_by_year()

857 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{1996: 358,
 1999: 82,
 2000: 296,
 2001: 70,
 2005: 121,
 2006: 4,
 2007: 1,
 2011: 39,
 2015: 29}

**Вывод:**

В 1996 был пик использования площадкой(предположительно) и пользователи поставили наибольшее количество оценок. Далее идут 2000 и 2005 год.

#### Какие оценки (0.5-5.0) встречаются чаще всего?

Метод `ratings.class_movies.dist_by_rating()` строит распределение значений рейтинга и возвращает {rating: count}, отсортировано по возрастанию rating.

In [11]:
%timeit -r1 -n1 ratings.class_movies.dist_by_rating()
ratings.class_movies.dist_by_rating()

421 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{0.5: 24,
 1.0: 39,
 1.5: 11,
 2.0: 57,
 2.5: 7,
 3.0: 253,
 3.5: 17,
 4.0: 292,
 4.5: 33,
 5.0: 267}

**Вывод:**

Самая популярная оценка 4.0

#### Какие фильмы получили больше всего оценок в нашей объединенной выборке?

Метод `ratings.class_movies.top_by_num_of_ratings(n)` считает количество записей по title в ratings.data (это уже join ratings+movies) и возвращает {title: count} для топ n по убыванию.

In [12]:
%timeit -r1 -n1 ratings.class_movies.top_by_num_of_ratings(10)
ratings.class_movies.top_by_num_of_ratings(10)

491 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'Usual Suspects, The (1995)': 4,
 'Pulp Fiction (1994)': 4,
 'Fugitive, The (1993)': 4,
 "Schindler's List (1993)": 4,
 'Batman (1989)': 4,
 'Silence of the Lambs, The (1991)': 4,
 'Fargo (1996)': 4,
 'Aladdin (1992)': 4,
 'Beauty and the Beast (1991)': 4,
 'Toy Story (1995)': 3}

**Вывод:**

В первой тысячи фильмов самое большое количество отзывов у 9 фильмов. И оно равно 4.

#### Какие фильмы имеют самый высокий средний рейтинг?

Метод `ratings.class_movies.top_by_ratings(n)` считает среднюю оценку по каждому title, округляет до 2 знаков и возвращает топ n {title: avg} по убыванию.

In [13]:
%timeit -r1 -n1 ratings.class_movies.top_by_ratings(10)
ratings.class_movies.top_by_ratings(10)

2.14 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'12 Angry Men (1957)': 5.0,
 'Adventures of Robin Hood, The (1938)': 5.0,
 'Alice in Wonderland (1951)': 5.0,
 'Aristocats, The (1970)': 5.0,
 'Back to the Future (1985)': 5.0,
 'Basic Instinct (1992)': 5.0,
 'Beautiful Thing (1996)': 5.0,
 'Bedknobs and Broomsticks (1971)': 5.0,
 'Before Sunrise (1995)': 5.0,
 'Bottle Rocket (1996)': 5.0}

**Вывод:**

Выше представлены топ 10 фильмов с самым высоким средним рейтингом.

Все они вышли до 2000.

#### Какие фильмы имеют самый высокий медианный рейтинг?

Метод `ratings.class_movies.top_by_ratings(10, "median")` собирает список оценок по каждому title, считает медиану, округляет до 2 знаков и возвращает топ n {title: median} по убыванию.

In [14]:
%timeit -r1 -n1 ratings.class_movies.top_by_ratings(10, "median")
ratings.class_movies.top_by_ratings(10, "median")

2.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'12 Angry Men (1957)': 5.0,
 'Adventures of Robin Hood, The (1938)': 5.0,
 'Alice in Wonderland (1951)': 5.0,
 'Aristocats, The (1970)': 5.0,
 'Back to the Future (1985)': 5.0,
 'Basic Instinct (1992)': 5.0,
 'Beautiful Thing (1996)': 5.0,
 'Bedknobs and Broomsticks (1971)': 5.0,
 'Before Sunrise (1995)': 5.0,
 'Bottle Rocket (1996)': 5.0}

**Вывод:**

Выше представлены фильмы с самым высоким медианным рейтингом.

Все они вышли до 2000 года.

#### Какие фильмы самые спорные по оценкам?

Метод `ratings.class_movies.top_controversial(n)` считает дисперсию оценок по каждому title (variance), округляет до 2 знаков и возвращает топ n {title: variance} по убыванию.

In [15]:
%timeit -r1 -n1 ratings.class_movies.top_controversial(10)
ratings.class_movies.top_controversial(10)

4.59 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'My Fair Lady (1964)': 5.06,
 "Schindler's List (1993)": 3.42,
 'Courage Under Fire (1996)': 3.06,
 'Usual Suspects, The (1995)': 2.42,
 'Circle of Friends (1995)': 2.25,
 'Dazed and Confused (1993)': 2.25,
 'Ghost (1990)': 2.25,
 'Hot Shots! Part Deux (1993)': 2.25,
 'Pulp Fiction (1994)': 2.19,
 'Tombstone (1993)': 2.0}

**Вывод:**

Самые спорные фильмы: My Fair Lady (1964), Schindler's List (1993), Courage Under Fire (1996).

## 5. Пользователи: активность, распределение средней/медианной оценки, топ по дисперсии

#### Сколько пользователей сделали 1 оценку, 2 оценки, 3 оценки и т.д.?

Метод `ratings.class_users.dist_by_num_of_ratings()` сначала считает количество оценок на каждого userId, затем строит распределение {кол-во оценок на пользователя: сколько пользователей}.

In [16]:
%timeit -r1 -n1 ratings.class_users.dist_by_num_of_ratings()
ratings.class_users.dist_by_num_of_ratings()

1.47 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{29: 1, 39: 1, 44: 1, 126: 1, 216: 1, 232: 1, 314: 1}

**Вывод:**

Пользователи довольно активно ставят оценки.

#### Какие средние оценки чаще встречаются у пользователей?

Метод `ratings.class_users.dist_by_ratings("average")` считает среднюю оценку каждого пользователя, округляет до 2 знаков и строит распределение {avg: count_users}.

In [17]:
%timeit -r1 -n1 ratings.class_users.dist_by_ratings("average")
ratings.class_users.dist_by_ratings("average")

613 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{2.44: 1, 3.35: 1, 3.49: 1, 3.56: 1, 3.64: 1, 3.95: 1, 4.37: 1}

**Вывод:**

Большинство пользователей в среднем оценивают фильмы оценкой ниже 4.0.

#### Какие медианные оценки чаще встречаются у пользователей?

Метод `ratings.class_users.dist_by_ratings("median")` считает медиану оценок каждого пользователя (устойчивее к выбросам) и строит распределение {median: count_users}.

In [18]:
%timeit -r1 -n1 ratings.class_users.dist_by_ratings("median")
ratings.class_users.dist_by_ratings("median")

630 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{0.5: 1, 3.0: 1, 4.0: 4, 5.0: 1}

**Вывод:**

У большинства пользователей медиана оценок равна 4.0.

#### Какие пользователи ставят самые разбросанные оценки?

Метод `ratings.class_users.top_controversial(n)` считает дисперсию оценок для каждого userId и возвращает топ n {userId: variance} по убыванию.

In [19]:
%timeit -r1 -n1 ratings.class_users.top_controversial(10)
ratings.class_users.top_controversial(10)

1.03 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{3: 4.26, 4: 1.72, 7: 1.65, 5: 0.96, 6: 0.72, 1: 0.64, 2: 0.63}

**Вывод:**

Пользователь с id=3 ставит самые разбросанные оценки.

## 6. Теги: популярные теги, самые длинные, самые многословные, поиск по слову


#### Какие теги состоят из наибольшего количества слов?

Метод `tags.most_words(n)` берет уникальные теги, считает число слов и возвращает top-n {tag: words_count}.

In [20]:
%timeit -r1 -n1 tags.most_words(10)
tags.most_words(10)

1.76 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'Something for everyone in this one... saw it without and plan on seeing it with kids!': 16,
 'the catholic church is the most corrupt organization in history': 10,
 'Oscar (Best Music - Original Score)': 6,
 'Everything you want is here': 5,
 'based on a true story': 5,
 'Guardians of the Galaxy': 4,
 'a clever chef rat': 4,
 'based on a book': 4,
 'based on a play': 4,
 'end of the world': 4}

**Вывод:**

Самый многословный тэг состоит из 16 слов. Но в среднем пользователи ограничиваются 4 словами.

#### Какие теги самые длинные по числу символов?

Метод `tags.longest(n)` берет уникальные теги и возвращает топ n список самых длинных строк.

In [21]:
%timeit -r1 -n1 tags.longest(10)
tags.longest(10)

1.08 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


['Something for everyone in this one... saw it without and plan on seeing it with kids!',
 'the catholic church is the most corrupt organization in history',
 'audience intelligence underestimated',
 'Oscar (Best Music - Original Score)',
 'assassin-in-training (scene)',
 'Everything you want is here',
 'Oscar (Best Cinematography)',
 'political right versus left',
 'representation of children',
 'Disney animated feature']

**Вывод:**

Есть пару длинных тэгов, но в среднем тэги около 25 символов и меньше.

#### Какие теги одновременно и длинные, и многословные?

Метод `tags.most_words_and_longest(n)` пересекает результаты most_words(n) и longest(n) и возвращает список общих тегов.

In [22]:
%timeit -r1 -n1 tags.most_words_and_longest(10)
tags.most_words_and_longest(10)

2.31 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


['Everything you want is here',
 'Oscar (Best Music - Original Score)',
 'Something for everyone in this one... saw it without and plan on seeing it with kids!',
 'the catholic church is the most corrupt organization in history']

**Вывод:**

Выше можно увидеть пересечение 2 множеств тэгов: самых длинных и самых многословных.

#### Какие теги пользователи используют чаще всего?

Метод `tags.most_popular(n)` считает частоту каждого тега и возвращает топ n {tag: count}.

In [23]:
%timeit -r1 -n1 tags.most_popular(10)
tags.most_popular(10)

447 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'funny': 15,
 'sci-fi': 14,
 'twist ending': 12,
 'dark comedy': 12,
 'atmospheric': 10,
 'superhero': 10,
 'comedy': 10,
 'action': 10,
 'suspense': 10,
 'Leonardo DiCaprio': 9}

**Вывод:**

Самые популярные тэги: funny, sci-fi, twist ending.

#### Какие теги содержат подстроку “comedy”?

Метод `tags.tags_with("...")` фильтрует уникальные теги по вхождению слова (без учета регистра) и возвращает отсортированный список.

In [24]:
%timeit -r1 -n1 tags.tags_with("comedy")
tags.tags_with("comedy")

495 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


['Comedy',
 'black comedy',
 'british comedy',
 'comedy',
 'dark comedy',
 'romantic comedy']

## 7. Links/IMDb: получение полей по фильмам через IMDb и расчет топов

#### Какие режиссеры самые продуктивные?


Метод `links.top_directors(n)` анализирует данные IMDb для фильмов из набора MovieLens, подсчитывает количество фильмов, снятых каждым режиссёром, и возвращает n самых продуктивных режиссёров в виде упорядоченного словаря, отсортированного по убыванию числа фильмов.

In [6]:
%timeit -r1 -n1 links.top_directors(10)
links.top_directors(10)

5min 17s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'Martin Scorsese': 2,
 'John Lasseter': 1,
 'Joe Johnston': 1,
 'Howard Deutch': 1,
 'Forest Whitaker': 1,
 'Charles Shyer': 1,
 'Michael Mann': 1,
 'Sydney Pollack': 1,
 'Peter Hewitt': 1,
 'Peter Hyams': 1}

**Вывод:**

Лидер по числу фильмов стал Martin Scorsese.

#### Какой фильм самый дорогой?

Метод `links.most_expensive(n)` анализирует бюджеты фильмов на основе данных IMDb, определяет фильмы с наибольшими затратами на производство и возвращает n самых дорогих фильмов в виде упорядоченного словаря, отсортированного по убыванию бюджета.

In [5]:
%timeit -r1 -n1 links.most_expensive(1)
links.most_expensive(1)

433 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'"Cutthroat Island"': '98000000'}

**Вывод:**

Одним из самых дорогих фильмов стал фильм Cutthroat Island (Остров головорезов)

#### Можно ещё посчитать цену минуты в каждом фильме

Метод `links.top_cost_per_minute(n)` вычисляет стоимость одной минуты фильма, деля производственный бюджет на продолжительность фильма. На основе этих данных метод формирует рейтинг и возвращает n фильмов с самой дорогой минутой экранного времени.

In [7]:
%timeit -r1 -n1 links.top_cost_per_minute(10)
links.top_cost_per_minute(10)

5min 1s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'"Cutthroat Island"': 790322.58,
 '"Pocahontas"': 679012.35,
 '"Jumanji"': 625000.0,
 '"Money Train"': 618181.82,
 '"Fair Game"': 549450.55,
 '"The American President"': 543859.65,
 '"The Indian in the Cupboard"': 468750.0,
 '"Broken Arrow"': 462962.96,
 '"GoldenEye"': 461538.46,
 '"Sabrina"': 456692.91}

**Вывод:**

Самая высокая стоимость минуты экранного времени наблюдается у фильма Cutthroat Island, заметно опережающего остальные фильмы рейтинга.

#### Если вдруг захотелось начать снимать фильмы после такого крутого обзора, то вот вам статистика по прибыли:

Метод `links.most_profitable(n)` анализирует данные IMDb о бюджете и кассовых сборах фильмов, вычисляет прибыль как разницу между сборами и бюджетом и возвращает n самых прибыльных фильмов в виде упорядоченного рейтинга.

In [18]:
links = mla.Links("links.csv", 100)
%timeit -r1 -n1 links.most_profitable(10)
links.most_profitable(10)

4min 50s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'"Toy Story"': 371157969,
 '"Se7en"': 296813591,
 '"GoldenEye"': 292194034,
 '"Pocahontas"': 291079773,
 '"Babe"': 224134910,
 '"Jumanji"': 197821940,
 '"Ace Ventura: When Nature Calls"': 182385533,
 '"The Bridges of Madison County"': 158016617,
 '"Dangerous Minds"': 156519401,
 '"Braveheart"': 141216216}

**Вывод:**

Самым прибыльным фильмом в подвыборке оказалась история игрушек, значительно опережающий остальные фильмы по разнице между кассовыми сборами и бюджетом.

#### Какие фильмы самые длинные?

Метод `links.longest(n)` анализирует продолжительность фильмов на основе данных IMDb и формирует рейтинг из n самых длинных фильмов, сортируя их по убыванию хронометража.

In [12]:
%timeit -r1 -n1 links.longest(10)
links.longest(10)

1.05 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


{'"Nixon"': '(192 min)',
 '"Casino"': '(178 min)',
 '"Braveheart"': '(178 min)',
 '"Les Misérables"': '(175 min)',
 '"Heat"': '(170 min)',
 '"Mr. Holland\'s Opus"': '(144 min)',
 '"Sense and Sensibility"': '(136 min)',
 '"The Bridges of Madison County"': '(135 min)',
 '"GoldenEye"': '(130 min)',
 '"12 Monkeys"': '(129 min)'}

**Вывод:**

Самым продолжительным фильмом в подвыборке оказался Hixon, превосходящий остальные фильмы по хронометражу.

Как работает метод парсинга?

In [13]:
movies_ids =  [1,2]
fields = ["title", "Director", "Budget", "Gross Worldwide", "Runtime"]
%timeit -r1 -n1 links.get_imdb(movies_ids, fields)
links.get_imdb(movies_ids, fields)

6.44 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


[[2,
  '"Jumanji"',
  'Joe Johnston',
  '$65,000,000 (estimated)',
  '$262,821,940',
  '(104 min)'],
 [1,
  '"Toy Story"',
  'John Lasseter',
  '$30,000,000 (estimated)',
  '$401,157,969',
  '(81 min)']]

## 8. ВЫВОДЫ:

В ходе работы был проведён анализ подвыборки данных MovieLens с использованием информации из IMDb. Были исследованы распределения фильмов и тегов, а также выполнен сравнительный анализ режиссёров, бюджетов, прибыли, продолжительности фильмов и стоимости минуты экранного времени.

Анализ показал, что в выбранной подвыборке отсутствуют ярко выраженные лидеры по количеству снятых фильмов: несколько режиссёров представлены двумя работами, в то время как большинство — единичными. При этом финансовые показатели фильмов значительно различаются: фильм Toy Story оказался самым прибыльным, а Cutthroat Island — лидером по стоимости одной минуты экранного времени, что подчёркивает несоответствие между затратами и коммерческим успехом.

Также было выявлено, что самые продолжительные фильмы не обязательно являются самыми дорогими или прибыльными, что говорит о слабой зависимости между хронометражом и финансовыми показателями. В целом, результаты анализа демонстрируют разнообразие характеристик фильмов и подчёркивают, что коммерческий успех определяется совокупностью факторов, а не одним параметром.